In [24]:
import sqlite3
#conn = sqlite3.connect('C:\sqlite\db\sample.db')
conn = sqlite3.connect('.\sample.db')
# Creates or opens a file called mydb with a SQLite3 DB


In [25]:
cursor = conn.cursor()


In [28]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:\sqllite\data.csv")

df.shape

(149299, 5)

In [29]:
df.dtypes

timestamp      object
user_id       float64
platform       object
course_id     float64
chapter_id    float64
dtype: object

In [30]:
df['course_id']=df['course_id'].astype(np.float).astype("Int32")
df['chapter_id']=df['chapter_id'].astype(np.float).astype("Int32")
df['user_id']=df['user_id'].astype(np.float).astype("Int32").astype(str)
df.shape
df.head()


,timestamp,user_id,platform,course_id,chapter_id
0,2020-03-07 00:00:05.448,3084148,Web,19,502
1,2020-03-07 00:00:06.822,2817123,Web,61,850
2,2020-03-07 00:00:07.521,3122446,Web,150,1840
3,2020-03-07 00:00:09.956,2519284,Web,12,303
4,2020-03-07 00:00:11.472,926242,Web,111,1706


In [31]:
df.dtypes

timestamp     object
user_id       object
platform      object
course_id      Int32
chapter_id     Int32
dtype: object

In [32]:
df.to_sql("course", conn, if_exists="replace")
#df.to_sql() function is used to write records stored in a DataFrame to a SQL database.
#first arg : name of db table
#conn:conection param



In [9]:
cursor.execute('''SELECT count(*) from course''')
user1 = cursor.fetchone() 
print(user1)

(149299,)


In [10]:
cursor.execute("PRAGMA table_info('course')").fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'timestamp', 'TEXT', 0, None, 0),
 (2, 'user_id', 'TEXT', 0, None, 0),
 (3, 'platform', 'TEXT', 0, None, 0),
 (4, 'course_id', 'INTEGER', 0, None, 0),
 (5, 'chapter_id', 'INTEGER', 0, None, 0)]

In [11]:
cursor.execute('''SELECT * from course''')
row1 = cursor.fetchone() 
print(row1)

(0, '2020-03-07 00:00:05.448', '3084148', 'Web', 19, 502)


In [12]:
cursor.execute('''select time(timestamp)   as part_of_day,user_id
from course limit 5''')
row1 = cursor.fetchall() 
print(row1)

[('00:00:05', '3084148'), ('00:00:06', '2817123'), ('00:00:07', '3122446'), ('00:00:09', '2519284'), ('00:00:11', '926242')]


In [13]:
cursor.execute('''select part_of_day,count(*) from (select case when time(timestamp) between '00:00:01' and '12:00:00' then 'Morning' 
when time(timestamp) between '12:00:01' and '17:00:00' then 'Afternoon' 
else 'Evening'
end as part_of_day,user_id
from course)a
group by part_of_day''')
row1 = cursor.fetchall() 
print(row1)

[('Afternoon', 31798), ('Evening', 55208), ('Morning', 62293)]


In [14]:
df1 = pd.read_sql_query('''select part_of_day,count(*) as cnt from (select case when time(timestamp) between '00:00:01' and '12:00:00' then 'Morning' 
when time(timestamp) between '12:00:01' and '17:00:00' then 'Afternoon' 
else 'Evening'
end as part_of_day,user_id
from course)a
group by part_of_day''', conn)
df1.head()

,part_of_day,cnt
0,Afternoon,31798
1,Evening,55208
2,Morning,62293


In [15]:
df2 = pd.read_sql_query('''select part_of_day,course_id,count(*) as cnt from (select case when time(timestamp) between '00:00:01' and '12:00:00' then 'Morning' 
when time(timestamp) between '12:00:01' and '17:00:00' then 'Afternoon' 
else 'Evening'
end as part_of_day,course_id 
from course)a
group by part_of_day,course_id
order by cnt desc''', conn)
df2.head()

,part_of_day,course_id,cnt
0,Morning,187.0,7243
1,Morning,150.0,6667
2,Evening,150.0,6317
3,Evening,187.0,5279
4,Afternoon,150.0,3572


In [16]:
df3 = pd.read_sql_query('''select part_of_day,chapter_id,count(*) as cnt from (select case when time(timestamp) between '00:00:01' and '12:00:00' then 'Morning' 
when time(timestamp) between '12:00:01' and '17:00:00' then 'Afternoon' 
else 'Evening'
end as part_of_day,chapter_id
from course)a
group by part_of_day,chapter_id
order by cnt desc''', conn)
df3.head()

,part_of_day,chapter_id,cnt
0,Morning,1921.0,1121
1,Morning,1924.0,1062
2,Morning,1922.0,857
3,Morning,1923.0,786
4,Evening,1924.0,753
